# Dados geoespaciais na prática
Agora que nós vimos os conceitos, técnicas e ferramentas para trabalhar com dados geoespaciais, vamos ver na prática como trabalhar com os dados. A idéia aqui é mostrar como pode ser fácil usar dados geoespaciais. 

- Leitura de dados geoespaciais 
- Limpeza dos dados
- Filtragem dos dados
- Análise e visualização dos dados
- Extração de conhecimento

In [ ]:
# Importando algumas bibliotecas para trabalharmos com os dados

# Pandas é uma biblioteca que permite criar dataframes, que são tabelas de dados.
# Nelas que nós armazenaremos os nossos dados enquanto trabalhamos com eles.
import pandas as pd

# Numpy é uma biblioteca para trabalhar com vetores/matrizes, e realizar cálculos
# sobre eles.
import numpy as np

# Matplotlib é uma biblioteca para a construção de gráficos simples
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 6]

# Seaborn é uma biblioteca para a construção de gráficos um pouco mais complexos
import seaborn as sns

# Folium é uma biblioteca para a construção de gráficos de dados geoespaciais
import folium

from IPython.display import display

# Plotly é uma biblioteca para construção de gráficos complexos, com suporte
# para gráficos geoespaciais
import plotly.express as px

# Shapely é uma biblioteca para a construção de formas geométricas, como Pontos,
# Linhas, e formas geométricas em geral
from shapely.geometry import Point

# Networkx é uma biblioteca para a construção de grafos
import networkx as nx
import osmnx as ox
ox.config(use_cache=True, log_console=True)

# Haversine é uma biblioteca para calcularmos a distância de haversine entre duas
# coordenadas geoespaciais
from haversine import haversine

# Para trabalharmos com datas e horas
from datetime import date

## 1. Leitura e 'conhecimento' breve dos dados
A seguir, vamos carregar os dados e dar uma olhada rápida para entendermos
suas características

In [ ]:
# Lendo o arquivo com os dados geoespaciais
dataset = pd.read_csv("new_york_taxi_data.csv")

In [ ]:
# Vamos ver como o nosso dataset se parece
dataset

Pela visualização acima, vemos que temos 400 mil linhas (400 mil viagens)
e 19 colunas, onde cada coluna representa uma característica das viagens

In [ ]:
# Vamos gerar uma amostra, para podermos trabalhar de forma mais rápida
# dataset = dataset.sample(100000)

In [ ]:
# Vamos checar as colunas do conjunto de dados
dataset.columns

## Entendendo as colunas:

- VendorID: Identificador do fornecedor da tecnologia de coleta dos dados das viagens
- tpep_pickup_datetime: Data e hora do momento no qual o taxímetro foi ligado
- tpep_dropoff_datetime: Data e hora do momento no qual o taxímetro foi desligado
- passenger_count: Número de passageiros
- trip_distance: Distância percorrida (em milhas) durante a viagem
- pickup_longitude: Coordenadas de longitude do início da viagem
- pickup_latitude: Coordenadas de latitude do início da viagem
- RatecodeID: 
- store_and_fwd_flag:
- dropoff_longitude:
- dropoff_latitude:
- payment_type:
- fare_amount:
- extra:
- mta_tax:
- tip_amount:
- tolls_amount:
- improvement_surcharge:
- total_amount:

Podemos apagar as colunas que não pretendemos usar em nossas análises.
Lembrem-se: ao trabalhar com grandes quantidades de dados, é preciso se atentar
ao uso eficiente dos recursos computacionais, especialmente quanto à memória

In [ ]:
# Vamos apagar as colunas que não queremos usar
del dataset['payment_type']
del dataset['mta_tax']
del dataset['improvement_surcharge']
del dataset['extra']
del dataset['store_and_fwd_flag']
del dataset['RatecodeID']
del dataset['VendorID']

## 2. Preparação dos dados (pré processamento, limpeza e filtragem)

In [ ]:
# Fazendo alguns preprocessamentos iniciais para trabalharmos com os dados

# A distância original está em milhas; Vamos convertê-la para quilômetros
dataset['trip_distance_km'] = dataset['trip_distance'] * 1.60934

# Convertendo o campo contendo a data e hora do início da viagem para um formato
# datetime (o formato original é string)
dataset['tpep_pickup_datetime'] = pd.to_datetime(dataset['tpep_pickup_datetime'])

# Convertendo o campo contendo a data e hora do fim da viagem para um formato
# datetime (o formato original é string)
dataset['tpep_dropoff_datetime'] = pd.to_datetime(dataset['tpep_dropoff_datetime'])

# Calculando a duração da viagem em minutos (tempo de fim - tempo de início)
dataset['trip_duration_min'] = (dataset['tpep_dropoff_datetime'] - dataset['tpep_pickup_datetime']).astype('timedelta64[m]')

# Calculando a hora e o dia da semana na qual a viagem ocorre
dataset['hour'] = dataset.tpep_pickup_datetime.dt.hour
dataset['dia_da_semana'] = dataset.tpep_dropoff_datetime.dt.weekday
# Segunda = 0, Domingo = 6


# Por fim, vamos ordenar os registros pelo momento de início da viagem
dataset = dataset.sort_values(by='tpep_pickup_datetime')

Com os dados carregados e pré processados, vamos visualizar 
algumas das features em busca de comportamentos
anormais; Assim, fazemos a limpeza dos dados:

In [ ]:
# Vamos visualizar a distribuição da quantidade de passageiros por viagem
fig = px.histogram(dataset, x='passenger_count')
fig.show()

In [ ]:
# Vamos visualizar a distribuição da distância total (em km) das viagens
figure = px.histogram(dataset, x='trip_distance')
figure.show()

In [ ]:
# Vamos visualizar a distribuição das durações das viagens (em minutos)
figure = px.histogram(dataset, x='trip_duration_min')
figure.show()

In [ ]:
# Por fim, vamos visualizar a distribuição dos custos das viagens (em dólares)
figure = px.histogram(dataset, x='total_amount')
figure.show()

In [ ]:
# Para comprovar, vamos analisar as viagens com comprimento maior que 5000km

irregular_trips_distance = dataset[dataset.trip_distance_km > 5000]
irregular_trips_distance

Só há uma viagem, e pela duração (9 minutos), a distância não faz sentido. Podemos descartá-la

In [ ]:
# Vamos ver também as viagens com duração maior que 24 horas
irregular_trips_hour = dataset[dataset.trip_duration_min > 24*60]
irregular_trips_hour

Só há uma viagem com duração de aproximadamente 26 dias; Também podemos descartá-la

In [ ]:
# Por fim, vamos ver as as viagens com custo maior que 200 dólares
irregular_trips_amount = dataset[dataset.total_amount >= 250]
irregular_trips_amount

São algumas viagens, várias com comportamento inesperado; Vamos removê-las também

In [ ]:
# Aplicando a limpeza sobre os dados:

# Para a duração
dataset = dataset[dataset.trip_duration_min <= 2*60]

# Para a distância
dataset = dataset[dataset.trip_distance_km <= 50]

# Para o custo final
dataset = dataset[dataset.total_amount < 250]
dataset = dataset[dataset.total_amount > 0]

In [ ]:
figure = px.histogram(dataset, x='trip_distance_km')
figure.show()

figure = px.histogram(dataset, x='trip_duration_min')
figure.show()

figure = px.histogram(dataset, x='total_amount')
figure.show()

### Em seguida, vamos aplicar filtros espaciais para excluir as viagens que ocorrem fora da ilha de Manhattan.

In [ ]:
# Vamos utilizar a biblioteca osmnx para carregar as geometrias correspondetes à area da ilha
manhattan = ox.geocode_to_gdf('Manhattan, New York')

In [ ]:
# Visualizando a geometria de Manhattan
ax = ox.project_gdf(manhattan).plot()

In [ ]:
# Checando se o ponto de origem está dentro de Manhattan
def check_origin_inside(trip):
    origin = Point(trip.pickup_longitude, trip.pickup_latitude)
    for shape in manhattan.geometry.values[0]:
        if shape.contains(origin):
            return True
    return False

# Checando se o ponto de destino está dentro de Manhattan
def check_destination_inside(trip):
    destination = Point(trip.dropoff_longitude, trip.dropoff_latitude)
    for shape in manhattan.geometry.values[0]:
        if shape.contains(destination):
            return True
    return False

In [ ]:
# Aplicando as funções e salvando os resultados
dataset['origin_inside_manhattan'] = dataset.apply(check_origin_inside, axis=1)
dataset['destination_inside_manhattan'] = dataset.apply(check_destination_inside, axis=1)

In [ ]:
# Vamos ver a proporção das viagens:

# Distribuição das viagens quanto à localização de origem
print("Proporção de viagens com origem dentro (True) e fora (False) de Manhattan")
print(dataset.origin_inside_manhattan.value_counts(normalize=True))

# Distribuição das viagens quanto à localização de destino
print("\nProporção de viagens com destino dentro (True) e fora (False) de Manhattan")
print(dataset.destination_inside_manhattan.value_counts(normalize=True))

In [ ]:
# Filtrando viagens localizadas totalmente em Manhattan
dataset = dataset[dataset.origin_inside_manhattan == True]
dataset = dataset[dataset.destination_inside_manhattan == True]

## 3. Visualização dos dados
Com os dados prontos, vamos visualizá-los. Existem diversas formas de visualizar
dados geoespaciais, de acordo com os objetivos

In [ ]:
# Vamos começar visualizando todas as viagens em nosso dataset

# Primeiro, tentaremos visualizar usando um gráfico de dispersão
plt.subplot(121)
plt.scatter(dataset.pickup_longitude, dataset.pickup_latitude)
plt.title("Inicio da viagem")

plt.subplot(122)
plt.scatter(dataset.dropoff_longitude, dataset.dropoff_latitude)
plt.title("Fim da viagem")
plt.show()

Não é muito funcional; São muitas viagens e os pontos se sobrepõem...
Podemos tentar a visualização utilizando um mapa de calor, assim os dados se agrupam de forma melhor e não
perdemos informações essenciais:

In [ ]:
# Vamos testar também um mapa de calor
figure = px.density_mapbox(dataset,
                           lat='pickup_latitude',
                           lon='pickup_longitude',
                           radius=10,
                           zoom=11,
                           mapbox_style="stamen-terrain")
                        
figure.show()

A distribuição das viagens é bem uniforme; Talvez o mapa de calor não seja a visualização mais adequada

In [ ]:
# Vamos testar então um grid

# Para os pontos de partida
px.density_heatmap(dataset,
                   x="pickup_longitude",
                   y="pickup_latitude",
                   nbinsy=200,
                   nbinsx=200)

In [ ]:
# Para os pontos de destino
px.density_heatmap(dataset,
                   x="dropoff_longitude",
                   y="dropoff_latitude",
                   nbinsy=200,
                   nbinsx=200)

O mapa de calor dá um resultado melhor: podemos ver a distribuição espacial das viagens, e os pontos
com maior concentração de viagens estão visíveis

In [ ]:
# O que tem no ponto com o maior número de partidas?
m = folium.Map(location=[40.7515, -73.9940], zoom_start=17)
folium.Marker(location=[40.7515, -73.9940],
              popup='Local de partida mais frequente').add_to(m)
m

In [ ]:
# O que tem no ponto com o maior número de viagens?
m = folium.Map(location=[40.7505, -73.9915], zoom_start=17)
folium.Marker(location=[40.7505, -73.9915],
              popup='Local de destino mais frequente').add_to(m)
m

Pelos dados, podemos afirmar a existência de um ponto de interesse das viagens de táxi: 
Madison Square Garden

In [ ]:
# Qual o tamanho ideal dos grãos do gráfico em grid?

# 10x10 grãos
figure = px.density_heatmap(dataset,
                            x="pickup_longitude",
                            y="pickup_latitude",
                            nbinsy=10,
                            nbinsx=10)
figure.show()

# 100x100 grãos
figure = px.density_heatmap(dataset,
                            x="pickup_longitude",
                            y="pickup_latitude",
                            nbinsy=100,
                            nbinsx=100)
figure.show()

# 1000x1000 grãos
figure = px.density_heatmap(dataset,
                            x="pickup_longitude",
                            y="pickup_latitude",
                            nbinsy=1000,
                            nbinsx=1000)
figure.show()

In [ ]:
# Podemos até mesmo adicionar outras variáveis ao nosso grid; nesse caso, o custo total das viagens
# Para isso, podemos escolher também uma função de agregação: valor utilizar a média
px.density_heatmap(dataset,
                   x="pickup_longitude",
                   y="pickup_latitude",
                   histfunc='avg',
                   z='total_amount',
                   nbinsy=1000,
                   nbinsx=1000)


Ao olhar o resultado do gráfico acima, podemos concluir que não há influência do local de partida sobre o preço final da viagem

Podemos analisar também a distribuição das viagens durante o tempo:

In [ ]:
# Vamos filtrar todas as viagens que ocorrem no dia 1o de Janeiro
day_sample1 = dataset[dataset.tpep_pickup_datetime.dt.date == date(2016, 1, 1)].copy()

In [ ]:
# Vamos filtrar também todas as viagens que ocorreram no dia 15 de Janeiro
day_sample2 = dataset[dataset.tpep_pickup_datetime.dt.date == date(2016, 1, 15)].copy()

In [ ]:
# Vamos visualizar a dispersão das viagens com o passar das horas durante o dia:

figure = px.scatter_mapbox(day_sample1,
                        lat='pickup_latitude',
                        lon='pickup_longitude',
                        animation_frame='hour',
                        color='trip_duration_min', # E se quisermos visualizar a distância ou tempo de viagem?
                        zoom=10,
                        title='Viagens no dia 1o de Janeiro')
figure.update_layout(mapbox_style="open-street-map")
figure.show()


figure = px.scatter_mapbox(day_sample2,
                        lat='pickup_latitude',
                        lon='pickup_longitude',
                        animation_frame='hour',
                        color='trip_duration_min', # E se quisermos visualizar a distância ou tempo de viagem?
                        zoom=10,
                        title='Viagens no dia 15 de Janeiro')
figure.update_layout(mapbox_style="open-street-map")
figure.show()

## 4. Aplicação dos dados
Agora que visualizamos os dados e fizemos análises iniciais, podemos propor aplicações para eles;

Vamos ver rapidamente 4 aplicações para nossos dados:
- Detecção de fraudes
- Detecção de Pontos de Interesse (PoI)
- Ridesharing
- Efeitos do tempo nas viagens

In [ ]:
def filtra_viagens(viagens, origem, destino, raio=.250):
    viagens.loc[:, "dist_origem"] = viagens.apply(lambda x: haversine((x.pickup_latitude,
                                                                       x.pickup_longitude),
                                                                      (origem[0],
                                                                       origem[1])), axis=1)
    
    viagens.loc[:, "dist_destino"] = viagens.apply(lambda x: haversine((x.dropoff_latitude,
                                                                        x.dropoff_longitude),
                                                                       (destino[0],
                                                                        destino[1])), axis=1)

    viagens_filtradas = dataset[(dataset.dist_origem < raio) & (dataset.dist_destino < raio)]
    return viagens_filtradas.copy()


###  4.1 Detecção de fraudes
Dado um ponto de origem O, um ponto de destino D:
- Qual a distância média das viagens entre esses dois pontos?
- Qual o custo médio das viagens?
- É possível detectar uma fraude por uma viagem com distância muito acima da média?
  - Ela possui custo acima da média também?

In [ ]:
# Vamos definir uma origem
origem = [40.7515, -73.9940]

# E um destino
destino = [40.7360, -73.9911]


# Filtrando as viagens
viagens = filtra_viagens(dataset, origem, destino)
                                                   
figure = px.histogram(viagens, x='total_amount')
figure.show()

figure = px.histogram(viagens, x='trip_distance')
figure.show()

figure = px.histogram(viagens, x='trip_duration_min')
figure.show()

figure = px.scatter(viagens, x='total_amount', y='trip_distance_km', color='tip_amount') #adicionar tips
figure.show()

### 4.2 Pontos de Interesse (PoI)
Conseguimos detectar pontos de interesse a partir das viagens? Como podemos fazê-lo? Dica: Podemos usar os volumes de viagens em determinadas regiões

In [ ]:
viagens_sexta = dataset[dataset.dia_da_semana == 4]
viagens_sexta = viagens_sexta.sort_values(by='hour')

fig = px.density_heatmap(viagens_sexta,
                        x='dropoff_latitude',
                        y='dropoff_longitude',
                        animation_frame='hour',
                        nbinsx=100,
                        nbinsy=100)
fig.update_layout(mapbox_style="open-street-map")


In [ ]:
# point = # LATITUDE, LONGITUDE
m = folium.Map(location=point, zoom_start=17)
folium.Marker(location=point,
              popup='Ponto de interesse').add_to(m)
m

### 4.3 Ridesharing
Se compartilharmos viagens, qual será o custo final para os passageiros?

In [ ]:
origem = [40.7515, -73.9940]
destino = [40.7360, -73.9911]

viagens = filtra_viagens(dataset, origem, destino, raio=.50)

viagens['quinto_de_hora'] = viagens.tpep_pickup_datetime.apply(lambda x: x.minute//10)

for gn, gd in viagens.groupby(by=[viagens.tpep_pickup_datetime.dt.month,
                                  viagens.tpep_pickup_datetime.dt.day,
                                  viagens.tpep_pickup_datetime.dt.hour,
                                  viagens.quinto_de_hora]):
    if len(gd) > 1:
        print(f"No mês {gn[0]}, dia {gn[1]}, hora {gn[2]} e intervalo de hora {gn[3]}, ocorreram as seguintes viagens:")
        display(gd[['tpep_pickup_datetime',
                   'passenger_count',
                   'total_amount',
                   'pickup_latitude',
                   'pickup_longitude',
                   'dropoff_latitude',
                   'dropoff_longitude']])
        
        confirmacao = input("Ver o próximo cenário? (s/n)")
        if confirmacao == 'n':
            break

### 4.4 Efeitos das condições climáticas
Para duas viagens semelhantes, os tempos são piores/custos maiores durante períodos de chuva/neve?

In [ ]:
origem = [40.7515, -73.9940]
destino = [40.7361, -73.9912]

viagens = filtra_viagens(dataset, origem, destino)

viagens['Com neve?'] = viagens.tpep_pickup_datetime.dt.month == 1

figure = px.scatter(viagens,
                    x='trip_duration_min',
                    y='trip_distance_km',
                    facet_col='Com neve?',
                    color='tip_amount')
figure.show()